# 3D Point Cloud Model of COVID-19

This project will create a 3D point cloud visualization model of the COVID-19 pandemic. The model will focus on modeling the global spatial patterns along with the statistics of different types of live cases for each country by thousands or even millions of georeferenced points.

## PREPARATION

Check python module/package installation. Uncomment the following code cell and run the specific `pip` statement first to install the missing python module/package if any module error occurs through `import` step.

In [ ]:
# pip install open3d

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import open3d as o3d
import urllib
import time
import os
from matplotlib import cm
from math import cos, sin, pi
from matplotlib import colors

## STEP1: LAND MASK DATA

*Notice: 
The following code cells are commented because the local file `landmask.geojson` has already created so that you can literally jump this step rather than run it again. But you are welcomed to uncomment and try again if needed.*

**1.1 create grid points**

In [ ]:
# # create 1-D arrays for lon and lat
# grid_lons = np.arange(-180, 180, 1)
# grid_lats = np.arange(-90, 90, 1)

# # cartesian product to pair each lon and lat
# grid_coords = [[a, b] for a in grid_lons for b in grid_lats]

**1.2 clip grid points with land only and add attributes**

In [ ]:
# # create a temporary dictionary for pandas dataframe
# coords_lon = []
# coords_lat = []
# dict_temp = {'lon': coords_lon, 'lat': coords_lat}

# for coord in grid_coords:
#     coords_lon.append(coord[0])
#     coords_lat.append(coord[1])

In [ ]:
# grid_points = pd.DataFrame.from_dict(dict_temp)
# world_polygons = gpd.read_file('World_Boundaries/World_Countries_(Generalized).shp')

# # generate the geometry from lon/lat columns of grid_points dataframe
# grid_points = gpd.GeoDataFrame(grid_points, geometry=gpd.points_from_xy(grid_points.lon, grid_points.lat))

# # keep the same coordinate reference system
# grid_points = grid_points.set_crs('EPSG:4326')
# world_polygons = world_polygons.to_crs('EPSG:4326')

In [ ]:
# # spatial join and clip grid_points with land mask only(remove sea mask)
# land = gpd.sjoin(grid_points, world_polygons, how='inner', op='intersects')
# land = land.drop(columns=['lon', 'lat', 'index_right'])

**1.3 write local GEOJSON file**

In [ ]:
# # write to local geojson
# land.to_file('landmask.geojson', driver='GeoJSON')

## STEP2: COVID-19 DATA

*Notice: 
The following code cells are commented because the local file `iso_coords.json` has already created so that you can literally jump this step rather than run it again. But you are welcomed to uncomment and try again if needed.*

**2.1 extract existing countries' `ISO` and the corresponding `geometry` from the `landmask.geojson` file**

In [ ]:
# # extract iso and coords value only from step1 output file
# iso_coords = {}

# with open('landmask.geojson', 'r') as f:
#     data = json.load(f)
    
# features = data['features']

# for feature in features:
#     iso = feature['properties']['ISO']
#     coords = feature['geometry']['coordinates']
    
#     # check key exist or not
#     if iso not in iso_coords.keys():
#         # create key then add the initial value
#         iso_coords[iso] = [coords]
#     else:
#         # add value directly
#         iso_coords[iso].append(coords)


In [ ]:
# # write iso and coords value into local file
# with open('iso_coords.json', 'w') as f:
#     json.dump(iso_coords, f)

**2.2 request available countries kept tracking on the COVID-19 API**

In [ ]:
# # query country names available from API
# url_countries = 'https://api.covid19api.com/countries'
# response = urllib.request.urlopen(url_countries)
# all_countries = json.loads(response.read())

**2.3 extract existing countries' `slug` for requesting COVID-19 data by country**


In [ ]:
# # extract iso-slug pairs for existing countries in landmask.geojson file
# # slug is the distinguished part for covid-19 data request url by country
# iso_slugs = {}

# for i in range(len(all_countries)):
#     iso = all_countries[i]['ISO2']
#     if iso in iso_coords.keys():
#         iso_slugs[iso] = all_countries[i]['Slug']

**2.4 start requesting each country's COVID-19 data and store in local JSON file**


In [ ]:
# # document the data request date
# action_date = time.strftime('%Y%m%d', time.gmtime())  # utc time

# # same part of all request urls
# request = 'https://api.covid19api.com/total/country/'

In [ ]:
# for iso, slug in iso_slugs.items():
#     # request covid-19 data by country
#     req = request + slug
#     res = urllib.request.urlopen(req)
#     data = json.loads(res.read())

#     # modify geometry response data
#     for i in range(len(data)):
#         # delete useless keys
#         data[i].pop('CountryCode')
#         data[i].pop('Province')
#         data[i].pop('City')
#         data[i].pop('CityCode')
#         data[i].pop('Lat')
#         data[i].pop('Lon')
        
#         # add keys
#         data[i]['ISO'] = iso
#         data[i]['Geometry'] = iso_coords[iso]

#     # store the modified covid-19 data locally
#     filename = f'data_by_country/{slug}_{action_date}.json' 
#     with open(filename, 'w') as f:
#         json.dump(data, f)


## STEP3: MODEL DEVELOPMENT

**3.1 read local covid-19 data for each country**

In [2]:
# store all coords and case(e.g. latest active cases) in the list
covid_coords = []

for file in os.listdir('data_by_country'):
    with open(f'data_by_country/{file}', 'r') as f:
        data = json.load(f)
        
        # empty list will occur list index eror
        if data:
            latest_data = data[-1]   # latest date
            # extract coords and active
            geom = latest_data['Geometry']
            active = latest_data['Active']  # active cases
            for coords in geom:
                coords.append(active)
                covid_coords.append(coords)   

**3.2 write 3D ply model**

In [3]:
def coords_conversion(lon, lat):
    ''' Convert the coordinates from Geodetic WGS84(LON/LAT) to
    3-dimensional Geocentric Cartesian Coordinates(X/Y/Z).     
    
    Attributes
    ----------
    lon: float
        Longitude of the point
    lat: float
        Latitude of the point
    
    Returns
    -------
    x: float
        values of x-axis for Cartesian Coordinates system
    y: float
        Values of y-axis for Cartesian Coordinates system
    z: float
        Values of z-axis for Cartesian Coordinates system. The point
        is on the surface of the globe.
      
    '''
    
    O = [0, 0, 0]   # center
    R = 6400        # radius
    S = 0.01        # scale
    
    
    # calculate the radian of the sphere
    rad_lat, rad_lon = lat * pi / 180, lon * pi / 180
    
    # calculate the cartesian coordiantes of each point
    x = O[0] + S * R * cos(rad_lat) * cos(rad_lon)
    y = O[1] + S * R * cos(rad_lat) * sin(rad_lon)
    z = O[2] + S * R * sin(rad_lat)


    return (x, y, z)

In [4]:
def point_color(x, min_val, max_val):                                     
    ''' Add colormap values for a set of normalized data.     
    
    Attributes
    ----------
    x: float
        A value from a set data to be processed a relative colormap value. 
    min_val: float
        The minimum observation value after leaving out the dirty data.
    max_val: float
        The maximum observation value after leaving out the dirty data.    
    
    Returns
    -------
    r: int
        Red value
    g: int
        Green value
    b: int
        Blue value
      
    '''
    
    # normalize the input data by relative min/max observation data
    ratio = (x-min_val) / (max_val-min_val)
    
    # calculate the value of color according to the ratio
    cmap = cm.YlOrRd  # a sequential colormap
    rgb = cmap(int(ratio*255))
    r = int(rgb[0]*255)
    g = int(rgb[1]*255)
    b = int(rgb[2]*255)

    return (r, g, b)

In [5]:
def write_model(path):
    ''' Contrust the formatted 3D Polygon File Format(PLY) with coordinates
    and colors for each point.
    
    Attribute
    ---------
    path: str
        The path and file name for the output PLY file.
    
    '''
    
    with open(path, 'w') as fw:
    # write headers with required format 
        fw.write('ply\nformat ascii 1.0\n')
        fw.write('element vertex %d\n' % len(model_coords))
        fw.write('property float x\n')
        fw.write('property float y\n')
        fw.write('property float z\n')

        if len(model_colors) == len(model_coords):
            fw.write('property uchar red\n')
            fw.write('property uchar green\n')
            fw.write('property uchar blue\n')

        fw.write('end_header\n')
        
        # write data
        if len(model_colors) == len(model_coords):
            for coord, color in zip(model_coords, model_colors):
                fw.write("%f %f %f %d %d %d\n" % (
                    coord[0],
                    coord[1],
                    coord[2],
                    color[0],
                    color[1],
                    color[2]
                    ))
        else:
            for coord in model_coords:
                fw.write("%f %f %f\n" % (
                    coord[0],
                    coord[1],
                    coord[2]
                    ))
        print('##### PLY model created #####')


In [6]:
covid_coords_t = np.transpose(covid_coords)
# a list stores all values of active cases only
covid_active = covid_coords_t[2]

# boxplot calculations
q1 = np.percentile(covid_active, [25, 50, 75])[0]  # lower quartile
q2 = np.percentile(covid_active, [25, 50, 75])[1]  # median
q3 = np.percentile(covid_active, [25, 50, 75])[2]  # upper quartile

qr = q3 - q1  # inter-qurtile range

# normal values range after leaving out dirty data
nor_min = q1 - 1.5 * qr
nor_max = q3 + 1.5 * qr


# min observation value
if nor_min < np.amin(covid_active):
    vmin = np.amin(covid_active)
else:
    vmin = nor_min

# max observation value
if nor_max > np.amax(covid_active):
    vmax = np.amax(covid_active)
else:
    vmax = nor_max


In [7]:
# store coords for model construction
model_coords = []
model_colors = []

# add covid data
model_coords.extend([ coords_conversion(lon, lat) for lon, lat, num in covid_coords ])
model_colors.extend([ point_color(num, vmin, vmax) for lon, lat, num in covid_coords ])

# construct model
write_model('model_latest_active.ply')

##### PLY model created #####


**3.3 display 3D model**

In [8]:
# read and display model
PC = o3d.io.read_point_cloud('model_latest_active.ply')
o3d.visualization.draw_geometries([PC])